# Generate Training Frames with Pretrained Model
This is a workflow to use a pretrained YOLO shark tracker to exrtact shark frames from a youtube video, double check them and use them as training frames for a more advanced model.

1. Run yolo with source=youtube video
2. Generate 1fps annotations from yolo results
3. Extract 1fps frames from video
4. Upload video and annotations on VIAME
5. Double check them [local]
6. Download new annotations [local]

In [11]:
from datetime import timedelta, datetime
from ultralytics import YOLO
import pandas as pd
import cv2
import os

In [10]:
video_name = 'gfp_jamaica1'
data_dir = '../raw/youtube_bruvs/' # directory containing video and annotation and frames
video_folder = data_dir + 'videos/'
annotation_folder = data_dir + 'annotations/'
video = video_folder + video_name + '.mp4'
annotation = annotation_folder + video_name + '.csv'
model_path = '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/best.pt'
frame_directory = data_dir + 'frames/'
ANNOTATIONS_FPS = 1
CONF_TRESHOLD = 0.2

## 1. Run yolo with source=youtube video

In [23]:
model = YOLO(model_path)
results = model.track(source=video, persist=True, conf=CONF_TRESHOLD)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/4821) /vol/biomedic3/bglocker/ugproj2324/fv220/dev/scripts/../../datasets/youtube_bruvs/gfp_jamaica1.mp4: 384x640 (no detections), 11.4ms
video 1/1 (2/4821) /vol/biomedic3/bglocker/ugproj2324/fv220/dev/scripts/../../datasets/youtube_bruvs/gfp_jamaica1.mp4: 384x640 (no detections), 6.7ms
video 1/1 (3/4821) /vol/biomedic3/bglocker/ugproj2324/fv220/dev/scripts/../../datasets/youtube_bruvs/gfp_jamaica1.mp4: 384x640 (no detections), 6.7ms
video

# 2. Generate Annotations
Must have the following format

```
# 1: Detection or Track-id	2: Video or Image Identifier	3: Unique Frame Identifier	4-7: Img-bbox(TL_x	TL_y	BR_x	BR_y)	8: Detection or Length Confidence	9: Target Length (0 or -1 if invalid)	10-11+: Repeated Species	Confidence Pairs or Attributes
# metadata	fps: 1	exported_by: "dive:python"	exported_time: "Mon Jan 15 15:56:15 2024"	Unnamed: 4	Unnamed: 5	Unnamed: 6	Unnamed: 7	Unnamed: 8	Unnamed: 9	Unnamed: 10
1	00:03.0	3	0	347	133	459	1	-1	shark	1
1	00:04.0	4	-6	314	127	426	1	-1	shark	1
1	00:05.0	5	2	456	228	612	1	-1	shark	1
1	00:06.0	6	67	301	410	466	1	-1	shark	1
1	00:07.0	7	211	275	555	436	1	-1	shark	1
1	00:08.0	8	260	304	648	465	1	-1	shark	1
```

In [28]:
# Get video length in seconds
vidcap = cv2.VideoCapture(video)
fps = vidcap.get(cv2.CAP_PROP_FPS)
frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps
print(f'Video duration: {duration} seconds')

Video duration: 160.86086086086087 seconds


In [92]:
def yolo_extract_tracks(results):
  """
  Given Raw yolo results extracts list of bbox, conf, track_id for each frame.
  :param results: List of predictions in the format results.bbox = [bbox_xyxy], [confidences], [track_ids]
  """
  bbox_xyxys = []
  confidences = []
  track_ids = []

  for i in range(len(results)):
    bbox_xyxy = []
    confidence = []
    track_id = []
    if results[i].boxes.id is not None:
      bbox_xyxy = results[i].boxes.xyxy.int().tolist()
      confidence = ((results[i].boxes.conf * 100).round() / 100).tolist()
      track_id = results[i].boxes.id.int().tolist()

    bbox_xyxys.append(bbox_xyxy)
    confidences.append(confidence)
    track_ids.append(track_id)

  return [bbox_xyxys, confidences, track_ids]


def get_fps_annotations(results, video_path):
    """
    ASSUME: Results are at video fps.

    Given results for each frame, extract only annotations for given fps.
    Returns list of annotations for each frame.
    {bbox_xyxys: [[frame1], [frame2], ...], track_ids: [[frame1], [frame2], ...]}
    """
    # get fps of video and tot frames
    vidcap = cv2.VideoCapture(video_path)
    video_fps = vidcap.get(cv2.CAP_PROP_FPS)
    tot_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
    assert len(results) == tot_frames

    video_duration = tot_frames / video_fps # seconds
    tot_annotation_frames = int(video_duration * ANNOTATIONS_FPS)

    track_predictions = yolo_extract_tracks(results)

    fps_results = {'bbox_xyxys': [], 'track_ids': [], 'confidences': [], 'frame_ids': []}

    # get annotations for each frame
    for frame_num in range(tot_annotation_frames):
        ### GET PRED FRAME TRACKS
        # Calculate the corresponding frame in the predictions
        pred_frame_index = int(round(frame_num * video_fps / ANNOTATIONS_FPS))
        assert pred_frame_index < tot_frames

        # Extract predicted data for the corresponding frame
        pred_bbox_xyxys = track_predictions[0][pred_frame_index]
        pred_confidences = track_predictions[1][pred_frame_index]
        pred_track_ids = track_predictions[2][pred_frame_index]

        fps_results['bbox_xyxys'].append(pred_bbox_xyxys)
        fps_results['track_ids'].append(pred_track_ids)
        fps_results['confidences'].append(pred_confidences)
        fps_results['frame_ids'].append(frame_num)
    
    assert len(fps_results['bbox_xyxys']) == tot_annotation_frames
    assert len(fps_results['track_ids']) == tot_annotation_frames
    assert len(fps_results['frame_ids']) == tot_annotation_frames
    assert len(fps_results['confidences']) == tot_annotation_frames

    return fps_results


def format_time(seconds):
    """
    Formats seconds to mm:ss:ms
    if 0 m, show 00 instead of 0
    ms should only be 2 digits
    """
    # Convert seconds to a timedelta
    td = timedelta(seconds=seconds)

    # Extract minutes, seconds, and milliseconds
    minutes = int(td.total_seconds() // 60)
    seconds = int(td.total_seconds() % 60)
    milliseconds = int(td.microseconds / 10000)  # Convert microseconds to milliseconds and round to 2 digits

    # Format the time string
    time_str = f"{minutes:02}:{seconds:02}:{milliseconds:02}"
    return time_str


def create_annotations_df(fps_cleaned_annotations):
  """
  Creates annotation df respecting required format:
  - columns: '# 1: Detection or Track-id',	'2: Video or Image Identifier' '3: Unique Frame Identifier'	'4-7: Img-bbox(TL_x'	'TL_y'	'BR_x'	'BR_y)'	'8: Detection or Length Confidence'	'9: Target Length (0 or -1 if invalid)'	'10-11+: Repeated Species	Confidence Pairs or Attributes'
  - rows: one row per bbox, ordered per track_id and frame. So first all bboxes for track_id 1 in frame order, then track_id 2, etc.
"""
  # Create df
  columns = ['# 1: Detection or Track-id',	'2: Video or Image Identifier', '3: Unique Frame Identifier',	'4-7: Img-bbox(TL_x',	'TL_y',	'BR_x',	'BR_y)',	'8: Detection or Length Confidence',	'9: Target Length (0 or -1 if invalid)',	'10-11+: Repeated Species', 'Confidence Pairs or Attributes']
  df = pd.DataFrame(columns=columns)

  # Add data
  for frame_num in range(len(fps_cleaned_annotations['bbox_xyxys'])):
      frame_bboxes = fps_cleaned_annotations['bbox_xyxys'][frame_num]
      frame_track_ids = fps_cleaned_annotations['track_ids'][frame_num]
      frame_id = fps_cleaned_annotations['frame_ids'][frame_num]
      frame_confidences = fps_cleaned_annotations['confidences'][frame_num]

      for bbox_num in range(len(frame_bboxes)):
          bbox = frame_bboxes[bbox_num]
          track_id = frame_track_ids[bbox_num]
          conf = frame_confidences[bbox_num]

          # Add row
          time = frame_id / ANNOTATIONS_FPS        
          time = format_time(time)  
          new_row = {
             '# 1: Detection or Track-id': track_id, 
             '2: Video or Image Identifier': time, 
             '3: Unique Frame Identifier': frame_id, 
             '4-7: Img-bbox(TL_x': bbox[0], 
             'TL_y': bbox[1], 
             'BR_x': bbox[2], 
             'BR_y)': bbox[3], 
             '8: Detection or Length Confidence': conf,
             '9: Target Length (0 or -1 if invalid)': -1,
             '10-11+: Repeated Species': 'shark',
             'Confidence Pairs or Attributes': 1
             }
          df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
          
  # Sort by track_id and frame_id
  df = df.sort_values(by=['# 1: Detection or Track-id', '3: Unique Frame Identifier'])
  df = df[columns]

  # Add a line below the header with the values '# metadata'	'fps: 1'	'exported_by: "dive:python"', 'exported_time: "Mon Jan 15 15:56:15 2024"'	'Unnamed: 4'	'Unnamed: 5'	'Unnamed: 6'	'Unnamed: 7'	'Unnamed: 8'	'Unnamed: 9'	'Unnamed: 10'
  current_date = datetime.now().strftime("%a %b %d %H:%M:%S %Y")
  metadata_row = {'# 1: Detection or Track-id': '# metadata', '2: Video or Image Identifier': f'fps: {ANNOTATIONS_FPS}', '3: Unique Frame Identifier': 'exported_by: "Filippo Varini"', '4-7: Img-bbox(TL_x': f'exported_time: "{current_date}"', 'TL_y': 'Unnamed: 4', 'BR_x': 'Unnamed: 5', 'BR_y)': 'Unnamed: 6', '8: Detection or Length Confidence': 'Unnamed: 7', '9: Target Length (0 or -1 if invalid)': 'Unnamed: 8', '10-11+: Repeated Species': 'Unnamed: 9', 'Confidence Pairs or Attributes': 'Unnamed: 10'}
  df = pd.concat([df, pd.DataFrame([metadata_row])], ignore_index=True)
  # make sure metadata row is at the beginning of the df (index 0)
  df = df.reindex([len(df)-1] + list(range(len(df)-1)))
  # remove indices
  df = df.reset_index(drop=True)
  

  return df
   
   

In [94]:
fps_cleaned_annotations = get_fps_annotations(results, video)
df = create_annotations_df(fps_cleaned_annotations)

/tmp/ipykernel_2207495/2147047372.py:129: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)


In [98]:
df.to_csv(annotation, index=False)

# 3. Extract ANNOTATION_FPS frames from video


In [13]:
frame_path = frame_directory + video_name + '/'
if not os.path.exists(frame_path):
    print(f'Creating frame directory {frame_path}')
    os.makedirs(frame_path)
else:
    print(f'Frame directory {frame_path} already exists. Writing there')

Frame directory ../raw/youtube_bruvs/frames/gfp_jamaica1/ already exists. Writing there


In [9]:
vidcap = cv2.VideoCapture(video)
success, image = vidcap.read()
count = 0
fps = vidcap.get(cv2.CAP_PROP_FPS)  # Get the frames per second (fps) of the video

while success:
  if count % round(fps / ANNOTATIONS_FPS) == 0:  # Save one frame per second
    cv2.imwrite(frame_directory + "frame%d.jpg" % count, image)  # Save frame as JPEG file
    print('Saved frame %d' % count)
  success, image = vidcap.read()
  count += 1



Saved frame 0
Saved frame 30
Saved frame 60
Saved frame 90
Saved frame 120
Saved frame 150
Saved frame 180
Saved frame 210
Saved frame 240
Saved frame 270
Saved frame 300
Saved frame 330
Saved frame 360
Saved frame 390
Saved frame 420
Saved frame 450
Saved frame 480
Saved frame 510
Saved frame 540
Saved frame 570
Saved frame 600
Saved frame 630
Saved frame 660
Saved frame 690
Saved frame 720
Saved frame 750
Saved frame 780
Saved frame 810
Saved frame 840
Saved frame 870
Saved frame 900
Saved frame 930
Saved frame 960
Saved frame 990
Saved frame 1020
Saved frame 1050
Saved frame 1080
Saved frame 1110
Saved frame 1140
Saved frame 1170
Saved frame 1200
Saved frame 1230
Saved frame 1260
Saved frame 1290
Saved frame 1320
Saved frame 1350
Saved frame 1380
Saved frame 1410
Saved frame 1440
Saved frame 1470
Saved frame 1500
Saved frame 1530
Saved frame 1560
Saved frame 1590
Saved frame 1620
Saved frame 1650
Saved frame 1680
Saved frame 1710
Saved frame 1740
Saved frame 1770
Saved frame 1800
Sa